In [ ]:
import pandas as pd
import folium
import json
import matplotlib.pyplot as plt
from io import BytesIO
import base64
from rapidfuzz import fuzz, process
from datetime import datetime 
from functools import lru_cache
import requests

# Словарь для хранения важных рабочих английских названий столбцов
# Важно иметь такие же названия в 1 строке в импортируемом csv (разрешены суффиксы после _)
column_names = {
    "region": "region",
    "total_population": "total_population"
}

prefix_mapping = {}

In [ ]:
# Импорт комбинированной таблицы
combined_csv_path = input("Введите путь к комбинированной таблице CSV: ").strip()
if not combined_csv_path:
    combined_csv_path = 'combined_table.csv'

# Загружаем данные с двумя строками заголовков
combined_data_df = pd.read_csv(combined_csv_path, sep=';', encoding='utf-8-sig', header=[0, 1])

# Разделяем заголовки на английские и русские
english_headers = combined_data_df.columns.get_level_values(0).str.lower()  # Все английские названия в нижний регистр
russian_headers = combined_data_df.columns.get_level_values(1)

# Создаем словарь маппинга между английскими и русскими названиями
header_mapping = dict(zip(english_headers, russian_headers))

# Переименовываем столбцы на английские названия
combined_data_df.columns = english_headers

# Приводим данные к нижнему регистру и нормализуем
combined_data_df[column_names["region"]] = combined_data_df[column_names["region"]].str.strip().str.lower().str.replace('ё', 'е')

# Загрузка GeoJSON данных
geojson_path = input("Введите путь к файлу GeoJSON: ").strip()
if not geojson_path:
    geojson_path = r'json\Voronezh_region.geojson'
with open(geojson_path, 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

# Центр карты (потом автоматизировать)
region_center_incsv = [51.67, 39.18]  # Указать центр области
m = folium.Map(location=region_center_incsv, zoom_start=8)

In [ ]:
# Функция для получения русского названия
def get_russian_name(english_name):
    # Проверяем, есть ли английское название напрямую в header_mapping
    if english_name in header_mapping:
        return header_mapping[english_name]

    # Если нет, ищем среди полных названий столбцов через prefix_mapping
    full_name = prefix_mapping.get(english_name)
    if full_name and full_name in header_mapping:
        return header_mapping[full_name]

    # Если ничего не найдено, возвращаем английское название
    return english_name

# Функция для группировки столбцов
def group_columns(columns):
    groups = {}
    prefix_mapping = {}  # Словарь для маппинга префиксов на полные названия столбцов

    for col in columns:
        if '_' in col:
            parts = col.split('_')
            last_part = parts[-1]  # Последняя часть после последнего подчёркивания
            prefix_parts = []

            # Извлекаем основной префикс, игнорируя года или числовые индексы
            for part in parts[:-1]:
                if not part.isdigit():  # Если часть не является числом, добавляем её в префикс
                    prefix_parts.append(part)
            prefix = '_'.join(prefix_parts)  # Префикс — всё до последней числовой части

            # Если последняя часть является числом (год или индекс)
            if last_part.isdigit():
                if prefix not in groups:
                    groups[prefix] = []
                groups[prefix].append(col)

                # Сохраняем маппинг префикса на полное название столбца
                if prefix not in prefix_mapping:
                    prefix_mapping[prefix] = col

                # Добавляем маппинг для обобщённого префикса в header_mapping
                if prefix not in header_mapping:
                    # Находим русское название для первого столбца в группе
                    full_name = col
                    if full_name in header_mapping:
                        russian_name = header_mapping[full_name]
                        # Удаляем суффикс из русского названия
                        russian_name_without_suffix = '_'.join(russian_name.split('_')[:-1])
                        header_mapping[prefix] = russian_name_without_suffix

    return groups, prefix_mapping

In [ ]:
# Функция для выбора ближайшего года/индекса
def get_closest_year_value(region_data, columns_group, current_year):
    closest_year = None
    closest_value = None
    for col in columns_group:
        try:
            year_or_index = int(col.split('_')[-1])  # Извлекаем последнюю часть (год или индекс)
            value = float(region_data[col].values[0])
            if closest_year is None or abs(year_or_index - current_year) < abs(closest_year - current_year):
                closest_year = year_or_index
                closest_value = value
        except ValueError:
            continue  # Пропускаем столбцы без числового суффикса
    return closest_value

# Функция для поиска соответствия между районами
def find_matching_region(target, candidates):
    target_normalized = target.strip().lower().replace('ё', 'е')
    if target_normalized in candidates:
        return target_normalized
    match = process.extractOne(
        target_normalized,
        candidates,
        scorer=fuzz.token_set_ratio,
        score_cutoff=85
    )
    if match:
        best_match = match[0]
        if any(best_match != candidate and best_match in candidate for candidate in candidates):
            return None
        return best_match
    return None

# Кэшируем результаты обработки данных для каждого района
@lru_cache(maxsize=None)
def process_district_data(district_name):
    match = find_matching_region(district_name, combined_data_df[column_names["region"]].unique())
    if not match:
        return None, 0, "Данные отсутствуют"

    # Группируем столбцы
    grouped_columns, prefix_mapping = group_columns(combined_data_df.columns)
    charts = []
    population = 0

    for group_prefix, columns in grouped_columns.items():
        # Получаем русское название группы
        group_name = get_russian_name(group_prefix)

        # Создаём диаграмму
        chart = create_chart(match, combined_data_df, tuple(columns), group_prefix)
        if chart:
            charts.append(chart)

        # Вычисляем population
        if column_names["total_population"] in group_prefix:
            current_year = datetime.now().year
            region_data = combined_data_df[combined_data_df[column_names["region"]] == match]
            population = get_closest_year_value(region_data, columns, current_year)

    # Объединяем все диаграммы в одну HTML-строку
    chart_html = ' '.join(charts) if charts else "Данные отсутствуют"
    return match, population, chart_html

# Функция стиля для GeoJSON
def style_function(feature):
    district_name = feature['properties']['district'].lower().replace('ё', 'е')
    match, population, chart_html = process_district_data(district_name)

    if match:
        # Создаём единый HTML
        popup_html = create_popup_html(district_name.capitalize(), chart_html)
        feature['properties']['popup_html'] = popup_html
        if population:
            feature['properties']['population'] = int(population)
            # Определяем цвет в зависимости от населения
            if population != 0 and population < 35000:
                color = 'green'
            elif population < 500000:
                color = 'orange'
            elif population > 500000:
                color = 'red'
            else:
                color = 'grey'
        else: # Если данных о популяции нет
            feature['properties']['population'] = "Данные отсутствуют"
            color = 'grey'
    else:
        color = 'grey'

    return {
        'fillColor': color,
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.4
    }

In [ ]:
# Функция для создания единого HTML
def create_popup_html(district_name, chart_html):
    html_template = f"""
    <div style="width: 100%; max-width: 800px; height: auto; margin: 0 auto; border: 1px solid #ddd; border-radius: 8px; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); overflow: hidden; font-family: Arial, sans-serif;">
        <div style="padding: 15px; background-color: #f9f9f9; border-bottom: 1px solid #ddd;">
            <h3 style="margin: 0; font-size: 18px; color: #333;">Район: {district_name}</h3>
        </div>
        <div style="padding: 15px; display: flex; flex-direction: column; align-items: center; max-height: 350px; min-width: 300px; overflow-y: auto; background-color: #fff;">
            {chart_html if chart_html else "<p style='color: #888; text-align: center;'>Данные отсутствуют</p>"}
        </div>
    </div>
    """
    return html_template

# Функция для создания диаграммы
def create_chart(region_name, combined_data_df, columns_group, group_prefix):
    print(f'Создание диаграммы для: {region_name}, группа: {group_prefix}')
    
    # Получаем русское название группы
    group_name = get_russian_name(group_prefix)
    if group_name == group_prefix:
        print(f"Предупреждение: Русское название не найдено для '{group_prefix}'. Используется английское название.")

    region_data = combined_data_df[combined_data_df[column_names["region"]] == region_name]
    if region_data.empty:
        return None

    years = []
    values = []
    for col in columns_group:
        try:
            year_or_index = int(col.split('_')[-1])  # Извлекаем последнюю часть (год или индекс)
            value = float(region_data[col].values[0])  # Получаем значение

            # Проверяем диапазон значений для процентов
            if "процент" in group_name.lower() and (value < 0 or value > 100):
                print(f"Некорректное значение {value} для {col} в регионе {region_name}")
                continue

            years.append(year_or_index)
            values.append(value)
        except ValueError:
            continue 

    if not years or not values:
        return None  # Если нет года или данных для построения диаграммы

    plt.figure(figsize=(3, 2))  # Размер графика
    bars = plt.bar(years, values, color=['blue', 'green', 'red'], alpha=0.6, label=group_name)
    plt.plot(years, values, color='black', marker='o', linewidth=2, label='Тренд')

    # Добавляем подписи над столбцами
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval + 0.5, f"{yval:.1f}", ha='center', va='bottom', fontsize=8)

    plt.title(f'{group_name}\n{region_name}', fontsize=10)
    plt.xlabel('Год/Индекс', fontsize=8)
    plt.ylabel(group_name, fontsize=8)
    plt.xticks(years, fontsize=8)
    plt.yticks(fontsize=8)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.legend(fontsize=8)

    buffer = BytesIO()
    plt.savefig(buffer, format='png', bbox_inches='tight')
    buffer.seek(0)
    image_base64 = base64.b64encode(buffer.read()).decode('utf-8')
    plt.close()

    return f'<img src="data:image/png;base64,{image_base64}" style="max-width: 100%; height: auto;">'

In [ ]:
# Добавление слоя для районов
district_layer = folium.FeatureGroup(name="Районы")
# Добавление GeoJson для районов на карту
folium.GeoJson(
    geojson_data,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['district', 'population'],
        aliases=['Район:', 'Население:'],
        localize=True
    ),
    popup=folium.GeoJsonPopup(
        fields=['popup_html'],
        aliases=[None],
        localize=True,
        labels=False,
        sticky=True,
        parse_html=True,
        max_width=800
    ),
    highlight_function=lambda x: {'weight': 3, 'color': 'black'},
).add_to(district_layer)

In [ ]:
# Добавление слоя для населённых пунктов
settlement_layer = folium.FeatureGroup(name="Населённые пункты", show=False)

# Функция для получения данных о населённых пунктах с OpenStreetMap
def fetch_settlements_from_overpass():
    print("Получение данных о населённых пунктах с OpenStreetMap...")
    overpass_url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"
    overpass_query = """
    [out:json][timeout:25];
    area["name"="Воронежская область"]["admin_level"="4"]->.region;
    (
      node["place"](area.region);
    );
    out body;
    >;
    out skel qt;
    """
    response = requests.post(overpass_url, data=overpass_query)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Ошибка при запросе к Overpass API: {response.status_code}")
        return None

# Функция для обработки данных о населённых пунктах
def process_settlements_data(settlements_data, combined_data_df):
    settlements = []
    for element in settlements_data.get("elements", []):
        if element.get("type") == "node" and "tags" in element:
            tags = element["tags"]
            settlement_name = tags.get("name", "").strip().lower().replace('ё', 'е')
            lat = element.get("lat")
            lon = element.get("lon")

            # Находим наиболее подходящее совпадение в CSV
            match = find_matching_region(settlement_name, combined_data_df[column_names["region"]].unique())
            if match:
                # Получаем данные о населении из CSV
                region_data = combined_data_df[combined_data_df[column_names["region"]] == match]
                population = None

                if not region_data.empty:
                    # Группируем столбцы для поиска данных о населении
                    grouped_columns, _ = group_columns(combined_data_df.columns)
                    population_columns = [col for col in grouped_columns if column_names["total_population"] in col]

                    if population_columns:
                        # Используем метод get_closest_year_value для выбора ближайшего года
                        current_year = datetime.now().year
                        closest_population = None
                        for population_group in population_columns:
                            closest_value = get_closest_year_value(region_data, grouped_columns[population_group], current_year)
                            if closest_value is not None:
                                closest_population = closest_value
                                break  # Берём первое найденное значение

                        if closest_population is not None:
                            try:
                                population = int(float(closest_population)) 
                            except ValueError:
                                population = None

                settlements.append({
                    "name": settlement_name.capitalize(),
                    "lat": lat,
                    "lon": lon,
                    "population": population
                })
    return settlements

# Добавление населённых пунктов на карту
def add_settlements_to_map(settlements, settlement_layer):
    for settlement in settlements:
        popup_html = f"<b>Населённый пункт:</b> {settlement['name']}<br>"
        tooltip_text = settlement['name']

        if settlement['population'] is not None:
            popup_html += f"<b>Население:</b> {settlement['population']}"
            tooltip_text += f" ({settlement['population']} чел.)"

        folium.Marker(
            location=[settlement["lat"], settlement["lon"]],
            tooltip=tooltip_text,
            popup=folium.Popup(popup_html, max_width=300)
        ).add_to(settlement_layer)

# Основной блок для работы с населёнными пунктами
settlements_data = fetch_settlements_from_overpass()
if settlements_data:
    print("Данные о населённых пунктах с OpenStreetMap успешно получены")
    settlements = process_settlements_data(settlements_data, combined_data_df)
    print("Добавление населённых пунктов на карту...")
    add_settlements_to_map(settlements, settlement_layer)
    print("Данные о населённых пунктах успешно добавлены на карту")
else:
    print("Не удалось получить данные о населённых пунктах.")

In [ ]:
# Добавляем слои на карту
district_layer.add_to(m)
settlement_layer.add_to(m)

# Добавляем элемент управления слоями
folium.LayerControl().add_to(m)

# Сохранение карты
output_map_path = input("Введите путь для сохранения карты (например, 'map.html'): ").strip()
if not output_map_path:
    output_map_path = 'map.html'
m.save(output_map_path)
display(m)
print(f"Карта успешно сохранена в файл: {output_map_path}")